In [ ]:
import torch as T
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
#importing libraries
import pandas as pd               
import io
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("https://raw.githubusercontent.com/xambert/Coding-Practice/main/Codes/df_afterPreprocessing.csv")

device = T.device("cpu")



In [ ]:
xcols = ["IN_PH","IN_TSS", "IN_BOD","IN_COD","IN_MPN"]
ycols = ["OUT_PH","OUT_DO","OUT_TSS"	,"OUT_COD"	,"OUT_BOD", "OUT_MPN"]

In [ ]:
def minmax(df, xcols):
  X = df[xcols].copy()
  for column in X.columns:
    X[column] = (X[column] - X[column].min())/(X[column].max() - X[column].min())
  return X

In [ ]:
new_data = data.copy()

In [ ]:
new_data[ycols] = minmax(new_data, ycols).copy()

In [ ]:
#import torch.nn.functional as F


In [ ]:
#shuffling data
train_df = new_data.sample(frac=0.8)
test_df = new_data.drop(train_df.index)


In [ ]:
def divide(df,xcol, ycol):
  p = T.Tensor(df[xcol].values.astype(np.float64))
  t = T.Tensor(df[ycol].values.astype(np.float64))
  t = T.reshape(t, (-1,1))
  return p,t
  

In [ ]:

class Net(T.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = T.nn.Linear(5, 10)  # 5-(10-10)-1
    self.hid2 = T.nn.Linear(10, 10)
    self.oupt = T.nn.Linear(10, 1)

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)


In [ ]:
model_0 = Net()

In [ ]:
def acc(model, dl, eps):
  cor , wro = 0, 0
  for X,Y in dl:
    with T.no_grad():
      outp = model(X)    
    abs_delta = np.abs(outp.item() - Y.item())
    max_allow = np.abs(eps * Y.item())
    if abs_delta < max_allow:
      cor += 1
    else:
      wro += 1
  acc = (cor * 100.0) / (cor + wro)
  return acc

In [ ]:
def fitandacc(model, opt,loss_fn, trdl, tedl, num_epochs):
  model.train()
  loss_list = []
  for epoch in range(num_epochs):
    cost = 0
    for xb, yb in trdl:
      y_pred = model(xb)
      loss = loss_fn(y_pred, yb)
      cost += loss
      #print(yb.item(), y_pred, loss)
      loss.backward()
      opt.step()
      opt.zero_grad()
    loss_list.append(cost.item()/len(trdl))
    if epoch %50==0:
      print("Epoch ", epoch, "Cost=", cost/len(trdl))
  model.eval()
  acctr = acc(model, trdl, 0.25)
  accte = acc(model, tedl, 0.25)
  return min(loss_list), acctr, accte

In [ ]:
#single hidden  layer
model_1 = nn.Sequential(nn.Linear(5,6), nn.ReLU(), nn.Linear(6,1)) 
model_2 = nn.Sequential(nn.Linear(5, 6), nn.Tanh(), nn.Linear(6,1))
model_3 = nn.Sequential(nn.Linear(5,6), nn.Sigmoid(), nn.Linear(6,1)) 
model_4 = nn.Sequential(nn.Linear(5,6), nn.ReLU(), nn.Linear(6,6),nn.ReLU(),nn.Linear(6,1)) 
model_5 = nn.Sequential(nn.Linear(5,6), nn.Sigmoid(), nn.Linear(6,6),nn.Sigmoid(),nn.Linear(6,1)) 
model_9 = nn.Sequential(nn.Linear(5,6), nn.Tanh(), nn.Linear(6,6),nn.Tanh(),nn.Linear(6,6),nn.Tanh(),nn.Linear(6,1))  
model_7 = nn.Sequential(nn.Linear(5,6), nn.ReLU(), nn.Linear(6,6),nn.ReLU(),nn.Linear(6,6),nn.ReLU(),nn.Linear(6,1)) 
model_8 = nn.Sequential(nn.Linear(5,6), nn.Sigmoid(), nn.Linear(6,6),nn.Sigmoid(),nn.Linear(6,6),nn.Sigmoid(),nn.Linear(6,1)) 
model_6 = nn.Sequential(nn.Linear(5,6), nn.Tanh(), nn.Linear(6,6),nn.Tanh(),nn.Linear(6,1))

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        T.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.00001)

In [ ]:
for i in range(10):
  mod = eval("model_"+str(i))
  mod.apply(init_weights)
  print(mod)

modelmap = { "model_1" : "5-6-1", "model_2" : "5-6-1", "model_3" : "5-6-1", "model_4": "5-6-6-1", "model_5" : "5-6-6-1", "model_6" : "5-6-6-1", "model_7" : "5-6-6-6-1", "model_8" : "5-6-6-6-1", "model_9" : "5-6-6-6-1"}
actmap = {"model_1" : "ReLU", "model_2": "TanH", "model_3":"Sigmoid", "model_4" : "ReLU", "model_5" : "TanH", "model_6" : "Sigmoid","model_7" : "ReLU", "model_9" : "TanH", "model_8" : "Sigmoid" }

In [ ]:
out_df = [pd.DataFrame(columns=["Model","Activation Function", "Cost", "Accuracy-Train", "Accuracy-Test"])]*6
for j,col in enumerate(ycols):
  ptrain, ttrain = divide(train_df, xcols,col)
  ptest, ttest = divide(test_df, xcols, col) 
  train_ds = TensorDataset(ptrain, ttrain)
  test_ds = TensorDataset(ptest, ttest)
  for i in range(1,10):
    train_dl = DataLoader(train_ds,batch_size=1,shuffle=True)
    test_dl = DataLoader(test_ds,batch_size=1,shuffle=True)
    mod = "model_"+str(i)
    opt = T.optim.Adam(eval(mod).parameters(),lr = 0.001)
    #opt = T.optim.Adam(net.parameters(),lr = 0.005)
    loss_fn = T.nn.MSELoss()
    #fn_cost = fit(eval(mod), opt, loss_fn, train_dl, pvalid, tvalid, 1)
    fn_cost, acctr, accte = fitandacc(eval(mod), opt, loss_fn, train_dl, test_dl, 300)
    test = pd.DataFrame([[modelmap[mod],actmap[mod], fn_cost, acctr, accte]], columns=["Model","Activation Function", "Cost", "Accuracy-Train", "Accuracy-Test"])
    out_df[j] = out_df[j].append(test,ignore_index=True)

In [ ]:
out_df

In [ ]:
for i,col in enumerate(ycols):
  print(col)
  print(out_df[i])
  out_df[i].to_csv(ycols[i]+".csv")

"Extras"

In [ ]:
out_df = [pd.DataFrame(columns=["Model", "Cost", "Accuracy-Train", "Accuracy-Test"])]*6
for j,col in enumerate([ycols[1]]):
  ptrain, ttrain = divide(train_df, xcols,col)
  ptest, ttest = divide(test_df, xcols, col) 
  train_ds = TensorDataset(ptrain, ttrain)
  test_ds = TensorDataset(ptest, ttest)
  for i in range(10):
    train_dl = DataLoader(train_ds,batch_size=1,shuffle=True)
    test_dl = DataLoader(test_ds,batch_size=1,shuffle=True)
    mod = "model_"+str(i)
    opt = T.optim.Adam(eval(mod).parameters(),lr = 0.005)
    #opt = T.optim.Adam(net.parameters(),lr = 0.001)
    loss_fn = T.nn.MSELoss()
    #fn_cost = fit(eval(mod), opt, loss_fn, train_dl, pvalid, tvalid, 1)
    fn_cost, acctr, accte = fitandacc(eval(mod), opt, loss_fn, train_dl, test_dl, 91)
    test = pd.DataFrame([[j+1, fn_cost, acctr, accte]], columns=["Model", "Cost", "Accuracy-Train", "Accuracy-Test"])
    out_df[1] = out_df[1].append(test,ignore_index=True)
print(ycols[1])
out_df[1]

In [ ]:
print(ycols[2])
out_df[2]

In [ ]:
print(ycols[3])
out_df[3]

In [ ]:
print(ycols[4])
out_df[4]

In [ ]:
print(ycols[5])
out_df[5]